# Optimization Framework using auto-diff to predict binding rates


## Import the necessary modules

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../")
import copy
from KineticAssembly_AD import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver
import networkx as nx
import torch
from torch import DoubleTensor as Tensor

## Setup Reaction Network
#### Read the corresponding input file and call the ReactionNetwork class

In [2]:
base_input = './former.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()


['default_assoc', 1.0]
New node added - Node index: 4 ; Node label: AB 
New node added - Node index: 5 ; Node label: AD 
Trying internal bonds
New node added - Node index: 6 ; Node label: BC 
New node added - Node index: 7 ; Node label: ABD 
Trying internal bonds
New node added - Node index: 8 ; Node label: CD 
New node added - Node index: 9 ; Node label: ABC 
New node added - Node index: 10 ; Node label: ACD 
New node added - Node index: 11 ; Node label: ABCD 
Trying internal bonds
New node added - Node index: 12 ; Node label: BCD 
Trying internal bonds
Trying internal bonds
Trying internal bonds
Trying internal bonds
Trying internal bonds
Trying internal bonds
Trying internal bonds
Trying internal bonds
Trying internal bonds
Trying internal bonds
Reaction Network Completed


## Checking reaction network
Looping over all network nodes to check if all species are created
Creating a dictionary for later reference. This dictionary holds the reactants as keys and values as the reaction index

In [3]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    print(n,"--",gtostr(rn.network.nodes[n]['struct']))
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid

print(uid_dict)

0 -- A
1 -- B
2 -- C
3 -- D
4 -- AB
5 -- AD
6 -- BC
7 -- ABD
8 -- CD
9 -- ABC
10 -- ACD
11 -- ABCD
12 -- BCD
{(0, 4): 0, (0, 5): 1, (0, 9): 13, (0, 10): 14, (0, 11): 17, (1, 4): 0, (1, 6): 2, (1, 7): 3, (1, 12): 15, (1, 11): 16, (2, 6): 2, (2, 8): 4, (2, 9): 5, (2, 10): 6, (2, 11): 7, (3, 5): 1, (3, 8): 4, (3, 7): 8, (3, 12): 9, (3, 11): 10, (4, 9): 5, (4, 7): 8, (4, 11): 11, (5, 7): 3, (5, 10): 6, (5, 11): 12, (6, 12): 9, (6, 11): 12, (6, 9): 13, (7, 11): 7, (8, 11): 11, (8, 10): 14, (8, 12): 15, (9, 11): 10, (10, 11): 16, (12, 11): 17}


## Set the initial parameter values 
For a hetero-trimer there are 6 reaction rates.
Also defines the Vectorized Rxn NEt class

In [4]:
new_kon = torch.zeros([rn._rxn_count], requires_grad=True).double()
new_kon = new_kon + Tensor([1])

update_kon_dict = {}
for edge in rn.network.edges:
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))


vec_rn = VectorizedRxnNet(rn, dev='cpu')
print(vec_rn.kon)

{'k_on': tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>), 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>), 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 1}
{'k_on': tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>), 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 13}
{'k_on': tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>), 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 14}
{'k_on': tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>), 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 17}
{'k_on': tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>), 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>), 'k_off': No

In [5]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off'],'uid':uid}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

A
B
C
D
AB
AD
BC
ABD
CD
ABC
ACD
ABCD
BCD
tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A'}, {'B'}), 'kon': tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>), 'score': tensor([-20.], dtype=torch.float64), 'koff': None, 'uid': 0}
tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'B'}, {'C', 'D'}), 'kon': tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>), 'score': tensor([-20.], dtype=torch.float64), 'koff': None, 'uid': 15}
tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A'}, {'C', 'D'}), 'kon': tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>), 'score': tensor([-20.], dtype=torch.float64), 'koff': None, 'uid': 14}
tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A'}, {'C', 'B'}), 'kon': tensor(1., dtype=torch.float64, grad_fn=<SelectBackward>), 'score': tensor([-20.], dtype=torch.float64), 'koff': None, 'uid': 13}
tensor(1., dtype=torch.float64, grad_fn=<Sele

## Using the optimizer ##

### Define an instance of the optimizer class
#### Input Arguments:

reaction_network : Input the vectorized rxn network

sim_runtime: The runtime of the kinetic simulation. Needs to be same as the time over the experimental reaction data.

optim_iterations: No. of iterations to run the optimization. Can start at low values(100) and increase depending upon memory usage.

learning_rate = The size of the gradient descent step for updating parameter values. Needs to be atleast (1e-3-1e-1)* min{parameter value}. If learning rate is too high, it can take a longer step and sometimes lead to negative value of parameters which is unphysical. Requires some trial runs to find the best value. 

device: cpu or gpu

method: Choose which pytorch based optimized to use for gradient descent - Adam or RMSprop

mom: Only for RMSprop method. Use momentum term during gradient descent. 


In [6]:
# import torch.utils.benchmark as benchmark
import time as time_mod

t1 = time_mod.perf_counter()

In [7]:
vec_rn.reset(reset_params=True)
optim = Optimizer(reaction_network=vec_rn,
                  sim_runtime=1,
                  optim_iterations=100,
                  learning_rate=1e-2,
                  device='cpu',method="RMSprop")


### Call the optimization method

#### Input arguments

conc_scale, conc_thresh, mod_bool, mod_factor = As defined for the VecSim class. 

max_thresh: Max. allowed values of parameters being updated. Beyond this maximum a penalty is imposed on the cost function. (Regularization)

max_yield: It is a control variable that is used to store the updated parameter values over all iterations for further analysis. The parameter values are stored only if the current yield exceed this max_yield. 

yield_species: Yield of the species being optimized(node index)

In [ ]:
optim.rn.update_reaction_net(rn)
optim.optimize(conc_scale=1e-1,conc_thresh=1e-1,mod_bool=True,mod_factor=10,max_thresh=1e8,max_yield=0,yield_species=-1)

Reaction Parameters before optimization: 
[Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64, requires_grad=True)]
Optimizer State: <bound method Optimizer.state_dict of RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)>
Using CPU
Yield on sim. iteration 0 was 13.5%.
current params: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=torch.float64)
tensor(0.1356, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
Yield on sim. iteration 1 was 15.1%.
current params: tensor([1.1000, 1.1000, 1.1000, 1.1000, 1.1000, 1.1000, 1.1000, 0.9000, 1.1000,
        1.1000, 0.9000, 0.9000, 0.9000, 1.1000, 1.1000, 1.1000, 0.9000, 0.9000],
       dtype=torch.float64)
tensor(0.1517, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
Yield on sim. iteration 2 was 16.3%.
current params: tensor([1.17

In [ ]:
t2 = time_mod.perf_counter()
print("Time taken for complete analysis: %.4f" %(t2-t1))

## Track the yield over optim iterations

In [ ]:
optim.plot_yield()

## Get all the parameter values

### This can be stored in a file for later analysis or used to find the best parameter value depending upon a condition. For e.g. the values that give a minimum error.

In [ ]:
yields= []
final_params=[]

final_t50 = []
final_t85 = []
final_t95 = []
final_t99 = []

for i in range(len(optim.final_yields)):
    yields.append(optim.final_yields[i])
    final_params.append(optim.final_solns[i])
    
    #Storing the different time points it reaches a particular yield threshold
    if optim.final_t85[i] == -1:
        final_t85.append(1) 
    else:
        final_t85.append(optim.final_t85[i]) 
    if optim.final_t95[i] == -1:
        final_t95.append(1)
    else:
        final_t95.append(optim.final_t95[i])

final_yield_arr = np.array(yields)
final_param_arr = np.array(final_params)
final_t85 = np.array(final_t85)
final_t95 = np.array(final_t95)

### Plotting the ratio of ktri vs kdim

In [ ]:
import matplotlib.pyplot as plt

mask_r = final_yield_arr > 0.5

#Calculate the ratio
ratio = final_param_arr[:,1]/final_param_arr[:,0]

#Normalize the time scale (t = t*conc*max_rate)
conc=vec_rn.initial_copies[0].item()
scale_time = final_t95[mask_r]*conc*np.max(final_param_arr[mask_r],axis=1)
#Calculate the y_per_time
y_per_time = 0.95/scale_time

fig,ax = plt.subplots(figsize=(8,6))
ax.plot(ratio,y_per_time,linestyle='',marker='o')
ax.set_ylabel("Efficiency",fontsize=20)
ax.set_xlabel("Ratio",fontsize=20)
ax.tick_params(labelsize=25)


In [ ]:
max_indx = np.argmax(y_per_time)
max_ratio = ratio[max_indx]
max_rates = final_param_arr[max_indx]
print("Ratio with maximum efficiency: ",max_ratio)

reaction_rates = np.zeros(rn._rxn_count)
counter=0
for cls,uids in vec_rn.rxn_class.items():
    for rid in uids:
        reaction_rates[rid]=max_rates[counter]
    counter+=1

print("Optimal Rates: ",list(reaction_rates))


In [ ]:
import pandas as pd
print(pd.__version__)